# Segmenting and Clustering Neighborhoods in Toronto

In [6]:
!pip install bs4

  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Using cached beautifulsoup4-4.8.2-py3-none-any.whl (106 kB)
  Using cached soupsieve-2.0-py2.py3-none-any.whl (32 kB)
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1278 sha256=54ba201df81823bf58c4ef38f30d94805001f200a4c5fe328e5122b733c7361b
  Stored in directory: c:\users\moata\appdata\local\pip\cache\wheels\0a\9e\ba\20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [131]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [11]:
Canada_M = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(Canada_M, "html")

In [123]:
PostalCode = []
Borough = []
Neighborhood = []

for data in soup.tbody.find_all('tr')[1:]:
    for td in data.find_all('td'):
        tdtxt=td.text.replace('\n','').replace(' /',',').replace(')','')
        if len(tdtxt[3:].split('(')[0]) > 0:
            PostalCode.append(tdtxt[:3])
            Borough.append(tdtxt[3:].split('(')[0])
            if len(tdtxt[3:].split('(')) == 1 :
                Neighborhood.append(tdtxt[3:].split('(')[0])
            else:
                Neighborhood.append(tdtxt[3:].split('(')[1])
            

In [129]:
alldata = {'PostalCode': PostalCode,
        'Borough': Borough,
        'Neighborhood': Neighborhood}
df = pd.DataFrame(alldata)
# Drop rows where Borough is 'Not assigned', then reset index
#df = df[df.Borough != 'Not assigned'].reset_index().drop('index', axis = 1)
df = df.groupby(df['PostalCode']).aggregate(sum).reset_index()
df.head(13)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [130]:
df.shape

(171, 3)